In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import (word_tokenize, sent_tokenize)

/tmp/ipykernel_150802/2726720222.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
df = pd.read_csv("./cnn_dailymail/train.csv")
df.drop(["id"], axis=1, inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   article     287113 non-null  object
 1   highlights  287113 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB


In [7]:
df.head()

,article,highlights
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [8]:
def remove_splchars(text):
    spl_chars = ["/", ":", "@", "*", "[", "]", "(", ")", "+", "-", "=", "|"]
    res = ""
    for i in text:
        if i in spl_chars:
            continue
        else:
            res += i
    return res

In [9]:
df["cleaned"]= np.NaN
df.head()

,article,highlights,cleaned
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ...",NaN
1,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...,NaN
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t...",NaN
3,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...,NaN
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...,NaN


In [10]:
df_size = df["article"].size

for i in range (0, df_size):
    temp = df["article"][i]
    cleaned = remove_splchars(temp)
    df["cleaned"][i] = cleaned

/tmp/ipykernel_5049/3260971211.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cleaned"][i] = cleaned
/tmp/ipykernel_5049/3260971211.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'By . Associated Press . PUBLISHED . 1411 EST, 25 October 2013 .  . UPDATED . 1536 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda pictured of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church 

KeyboardInterrupt: 

In [ ]:
df.to_pickle("cleaned.pkl")

In [3]:
import pandas as pd
import numpy as np
from nltk.tokenize import (word_tokenize, sent_tokenize)

In [2]:
df = pd.read_pickle("./cleaned.pkl")

In [7]:
df.drop("article", axis=1, inplace=True)

In [8]:
df.head()

,highlights,cleaned
0,"Bishop John Folda, of North Dakota, is taking ...","By . Associated Press . PUBLISHED . 1411 EST, ..."
1,Criminal complaint: Cop used his role to help ...,CNN Ralph Mata was an internal affairs lieute...
2,"Craig Eccleston-Todd, 27, had drunk at least t...",A drunk driver who killed a young woman in a h...
3,Nina dos Santos says Europe must be ready to a...,CNN With a breezy sweep of his pen President ...
4,Fleetwood top of League One after 2-0 win at S...,Fleetwood are the only team still to have a 10...


In [10]:
df["tokens"] = np.NaN

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   highlights  287113 non-null  object
 1   cleaned     287113 non-null  object
 2   tokens      1000 non-null    object
dtypes: object(3)
memory usage: 6.6+ MB


In [ ]:
df_size = df["cleaned"].size

for i in range (0, df_size):
    temp = df["article"][i]
    cleaned = word_tokenize(temp, 'english', preserve_line=True)
    df["cleaned"][i] = cleaned


In [18]:
def batching(df):
    batch_size = 1000
    temp = 0
    steps = np.ceil((df["cleaned"].size)/batch_size)
    while(steps>=0):
        df_temp = df.iloc[temp*batch_size:(temp+1)*batch_size, 1]
        yield df_temp
        temp+=1
        steps-=1 


In [30]:
count = 0

for i in batching(df):
    temp = i.transform(lambda x: word_tokenize(x, "english", preserve_line=True))
    # temp = word_tokenize(i, "english", preserve_line=True)
    df.iloc[count*1000:(count+1)*1000, 2] = temp 
    count += 1